# Material3 Panel Migration Project
This notebook guides through the process of migrating `CollectionPropertyPanel` and `ImagePropertyPanel` to Material3 with localization support.

## Setup Environment
First, let's ensure we have the necessary dependencies and project structure.

In [ ]:
# Check Flutter version and required packages
!flutter --version
!flutter pub get

# Material 3 Panel Migration with Localization
This notebook outlines the process of migrating CollectionPropertyPanel and ImagePropertyPanel to Material 3 with localization support.

## Setup Environment
First, we need to ensure we have all the necessary dependencies for Material 3 and localization support.

In [ ]:
# Check Flutter SDK version and update if needed
!flutter --version
!flutter upgrade

# Ensure Material 3 dependencies are added to pubspec.yaml
print("Make sure the following are in your pubspec.yaml:")
print("""
dependencies:
  flutter:
    sdk: flutter
  flutter_localizations:
    sdk: flutter
  intl: ^0.18.0
""")

## Project Structure
Let's understand the file structure we need to modify:
- Create new files with 'm3_' prefix
- Update the l10n files for localization

In [ ]:
# List current files to understand the structure
import os

project_path = os.getcwd()
print(f"Project path: {project_path}")

# Find relevant panel files
!find . -name "*property_panel*.dart" -type f

## Migrate CollectionPropertyPanel
We'll now convert the CollectionPropertyPanel to use Material 3 design principles and extract all hardcoded strings for localization.

In [ ]:
# Create a new file for Material 3 version
collection_panel_file = "lib/widgets/m3_collection_property_panel.dart"

collection_panel_code = """
import 'package:flutter/material.dart';
import 'package:flutter_gen/gen_l10n/app_localizations.dart';

class M3CollectionPropertyPanel extends StatefulWidget {
  final List<dynamic> collection;
  final Function(int, dynamic) onItemChanged;
  final Function(dynamic) onItemAdded;
  final Function(int) onItemRemoved;
  
  const M3CollectionPropertyPanel({
    Key? key,
    required this.collection,
    required this.onItemChanged,
    required this.onItemAdded,
    required this.onItemRemoved,
  }) : super(key: key);

  @override
  State<M3CollectionPropertyPanel> createState() => _M3CollectionPropertyPanelState();
}

class _M3CollectionPropertyPanelState extends State<M3CollectionPropertyPanel> {
  @override
  Widget build(BuildContext context) {
    final l10n = AppLocalizations.of(context)!;
    
    return Card(
      elevation: 0, // Material 3 uses less elevation
      shape: RoundedRectangleBorder(
        borderRadius: BorderRadius.circular(12), // M3 uses larger border radius
        side: BorderSide(color: Theme.of(context).colorScheme.outline), // M3 outline
      ),
      child: Padding(
        padding: const EdgeInsets.all(16.0),
        child: Column(
          crossAxisAlignment: CrossAxisAlignment.start,
          children: [
            Text(
              l10n.collectionItems,
              style: Theme.of(context).textTheme.titleMedium,
            ),
            const SizedBox(height: 8),
            ...List.generate(
              widget.collection.length,
              (index) => _buildCollectionItem(context, index, l10n),
            ),
            const SizedBox(height: 16),
            FilledButton.icon(
              icon: const Icon(Icons.add),
              label: Text(l10n.addItem),
              onPressed: () => widget.onItemAdded(null),
            ),
          ],
        ),
      ),
    );
  }

  Widget _buildCollectionItem(BuildContext context, int index, AppLocalizations l10n) {
    final item = widget.collection[index];
    
    return Card(
      margin: const EdgeInsets.only(bottom: 8.0),
      child: ListTile(
        title: Text('${l10n.item} #${index + 1}'),
        subtitle: Text(item.toString()),
        trailing: IconButton(
          icon: const Icon(Icons.delete_outline),
          tooltip: l10n.removeItem,
          onPressed: () => widget.onItemRemoved(index),
        ),
        onTap: () => _editItem(index, item),
      ),
    );
  }

  void _editItem(int index, dynamic item) {
    // Implement item editing logic here
    widget.onItemChanged(index, item);
  }
}
"""

# Write the new file
with open(collection_panel_file, "w") as f:
    f.write(collection_panel_code)

print(f"Created {collection_panel_file}")

## Migrate ImagePropertyPanel
Now, let's migrate the ImagePropertyPanel to Material 3 and add localization support.

In [ ]:
# Create a new file for Material 3 version
image_panel_file = "lib/widgets/m3_image_property_panel.dart"

image_panel_code = """
import 'dart:io';
import 'package:flutter/material.dart';
import 'package:flutter_gen/gen_l10n/app_localizations.dart';

class M3ImagePropertyPanel extends StatelessWidget {
  final String? imagePath;
  final Function(String?) onImageSelected;
  final double imageHeight;
  final double imageWidth;
  
  const M3ImagePropertyPanel({
    Key? key,
    this.imagePath,
    required this.onImageSelected,
    this.imageHeight = 200,
    this.imageWidth = double.infinity,
  }) : super(key: key);

  @override
  Widget build(BuildContext context) {
    final l10n = AppLocalizations.of(context)!;
    final colorScheme = Theme.of(context).colorScheme;
    
    return Card(
      elevation: 0, // Material 3 uses less elevation
      shape: RoundedRectangleBorder(
        borderRadius: BorderRadius.circular(12), // M3 uses larger border radius
        side: BorderSide(color: colorScheme.outline), // M3 outline
      ),
      child: Padding(
        padding: const EdgeInsets.all(16.0),
        child: Column(
          crossAxisAlignment: CrossAxisAlignment.start,
          children: [
            Text(
              l10n.imageProperty,
              style: Theme.of(context).textTheme.titleMedium,
            ),
            const SizedBox(height: 16),
            _buildImagePreview(context, l10n),
            const SizedBox(height: 16),
            Row(
              mainAxisAlignment: MainAxisAlignment.spaceEvenly,
              children: [
                FilledButton.icon(
                  icon: const Icon(Icons.photo_library),
                  label: Text(l10n.chooseFromGallery),
                  onPressed: () => _selectImage(context, 'gallery'),
                ),
                OutlinedButton.icon(
                  icon: const Icon(Icons.camera_alt),
                  label: Text(l10n.takePhoto),
                  onPressed: () => _selectImage(context, 'camera'),
                ),
              ],
            ),
            if (imagePath != null)
              Padding(
                padding: const EdgeInsets.only(top: 16.0),
                child: TextButton.icon(
                  icon: Icon(Icons.delete_outline, color: colorScheme.error),
                  label: Text(
                    l10n.removeImage,
                    style: TextStyle(color: colorScheme.error),
                  ),
                  onPressed: () => onImageSelected(null),
                ),
              ),
          ],
        ),
      ),
    );
  }

  Widget _buildImagePreview(BuildContext context, AppLocalizations l10n) {
    if (imagePath == null) {
      return Container(
        height: imageHeight,
        width: imageWidth,
        decoration: BoxDecoration(
          color: Theme.of(context).colorScheme.surfaceVariant,
          borderRadius: BorderRadius.circular(8),
        ),
        child: Center(
          child: Column(
            mainAxisAlignment: MainAxisAlignment.center,
            children: [
              Icon(
                Icons.image_outlined,
                size: 48,
                color: Theme.of(context).colorScheme.onSurfaceVariant,
              ),
              const SizedBox(height: 8),
              Text(
                l10n.noImageSelected,
                style: TextStyle(
                  color: Theme.of(context).colorScheme.onSurfaceVariant,
                ),
              ),
            ],
          ),
        ),
      );
    }

    return ClipRRect(
      borderRadius: BorderRadius.circular(8),
      child: Image.file(
        File(imagePath!),
        height: imageHeight,
        width: imageWidth,
        fit: BoxFit.cover,
      ),
    );
  }

  void _selectImage(BuildContext context, String source) async {
    // Implement image picking logic here
    // This is just a placeholder
    if (source == 'gallery') {
      // Pick from gallery
      onImageSelected('/path/to/image.jpg');
    } else {
      // Take photo
      onImageSelected('/path/to/camera_image.jpg');
    }
  }
}
"""

# Write the new file
with open(image_panel_file, "w") as f:
    f.write(image_panel_code)

print(f"Created {image_panel_file}")

## Generate l10n Entries
Now we need to create or update the localization files to include all strings we've extracted from the panels.

In [ ]:
# Create the l10n arb files directory if it doesn't exist
l10n_dir = "lib/l10n"
os.makedirs(l10n_dir, exist_ok=True)

# Create the main English localization file
en_arb_file = f"{l10n_dir}/app_en.arb"

arb_content = """
{
  "@@locale": "en",
  
  "collectionItems": "Collection Items",
  "@collectionItems": {
    "description": "Title for the collection items panel"
  },
  
  "addItem": "Add Item",
  "@addItem": {
    "description": "Button to add a new item to the collection"
  },
  
  "item": "Item",
  "@item": {
    "description": "Label for an item in the collection"
  },
  
  "removeItem": "Remove Item",
  "@removeItem": {
    "description": "Tooltip for the button to remove an item from the collection"
  },
  
  "imageProperty": "Image",
  "@imageProperty": {
    "description": "Title for the image property panel"
  },
  
  "chooseFromGallery": "Gallery",
  "@chooseFromGallery": {
    "description": "Button to choose an image from the gallery"
  },
  
  "takePhoto": "Camera",
  "@takePhoto": {
    "description": "Button to take a photo with the camera"
  },
  
  "removeImage": "Remove Image",
  "@removeImage": {
    "description": "Button to remove the current image"
  },
  
  "noImageSelected": "No image selected",
  "@noImageSelected": {
    "description": "Text shown when no image is selected"
  }
}
"""

# Write the English ARB file
with open(en_arb_file, "w") as f:
    f.write(arb_content)

print(f"Created {en_arb_file}")

# Create a Spanish localization file as an example of additional language support
es_arb_file = f"{l10n_dir}/app_es.arb"

arb_content_es = """
{
  "@@locale": "es",
  
  "collectionItems": "Elementos de la Colección",
  "addItem": "Añadir Elemento",
  "item": "Elemento",
  "removeItem": "Eliminar Elemento",
  "imageProperty": "Imagen",
  "chooseFromGallery": "Galería",
  "takePhoto": "Cámara",
  "removeImage": "Eliminar Imagen",
  "noImageSelected": "No hay imagen seleccionada"
}
"""

# Write the Spanish ARB file
with open(es_arb_file, "w") as f:
    f.write(arb_content_es)

print(f"Created {es_arb_file}")

## Update pubspec.yaml for Localization
We need to update the pubspec.yaml file to enable localization support.

In [ ]:
# Create code to update pubspec.yaml
pubspec_update = """
# Add the following to your pubspec.yaml:

flutter:
  # ... existing flutter section ...
  
  generate: true # Add this for localization generation
  
flutter_intl:
  enabled: true
  
# Then add these dependencies if not already present:
# dependencies:
#   flutter_localizations:
#     sdk: flutter
#   intl: ^0.18.0
"""

print(pubspec_update)

## Create l10n.yaml for Localization Configuration
Create a configuration file for the Flutter localization system.

In [ ]:
# Create the l10n.yaml file in the project root
l10n_yaml_file = "l10n.yaml"

l10n_yaml_content = """
arb-dir: lib/l10n
template-arb-file: app_en.arb
output-localization-file: app_localizations.dart
nullable-getter: false
"""

# Write the l10n.yaml file
with open(l10n_yaml_file, "w") as f:
    f.write(l10n_yaml_content)

print(f"Created {l10n_yaml_file}")

## Verify Localization Integration
Now we need to make sure the application is configured correctly to use our localized Material 3 panels.

In [ ]:
# Create code sample for main.dart to show proper configuration
main_dart_sample = """
import 'package:flutter/material.dart';
import 'package:flutter_localizations/flutter_localizations.dart';
import 'package:flutter_gen/gen_l10n/app_localizations.dart';

void main() {
  runApp(const MyApp());
}

class MyApp extends StatelessWidget {
  const MyApp({Key? key}) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Material 3 Demo',
      theme: ThemeData(
        useMaterial3: true,
        colorScheme: ColorScheme.fromSeed(seedColor: Colors.blue),
      ),
      // Configure localization
      localizationsDelegates: const [
        AppLocalizations.delegate,
        GlobalMaterialLocalizations.delegate,
        GlobalWidgetsLocalizations.delegate,
        GlobalCupertinoLocalizations.delegate,
      ],
      supportedLocales: const [
        Locale('en'), // English
        Locale('es'), // Spanish
      ],
      home: const HomePage(),
    );
  }
}

class HomePage extends StatefulWidget {
  const HomePage({Key? key}) : super(key: key);

  @override
  State<HomePage> createState() => _HomePageState();
}

class _HomePageState extends State<HomePage> {
  List<String> _collection = ['Item 1', 'Item 2', 'Item 3'];
  String? _imagePath;

  @override
  Widget build(BuildContext context) {
    final l10n = AppLocalizations.of(context)!;
    
    return Scaffold(
      appBar: AppBar(
        title: Text(l10n.appTitle),
      ),
      body: Padding(
        padding: const EdgeInsets.all(16.0),
        child: ListView(
          children: [
            M3CollectionPropertyPanel(
              collection: _collection,
              onItemChanged: (index, newValue) {
                setState(() {
                  _collection[index] = newValue;
                });
              },
              onItemAdded: (item) {
                setState(() {
                  _collection.add('New Item ${_collection.length + 1}');
                });
              },
              onItemRemoved: (index) {
                setState(() {
                  _collection.removeAt(index);
                });
              },
            ),
            const SizedBox(height: 16),
            M3ImagePropertyPanel(
              imagePath: _imagePath,
              onImageSelected: (path) {
                setState(() {
                  _imagePath = path;
                });
              },
            ),
          ],
        ),
      ),
    );
  }
}
"""

print("Sample main.dart to verify integration:")
print(main_dart_sample)

## Testing the Migration
Instructions for testing the migrated panels and localization.

In [ ]:
# Create testing instructions
test_instructions = """
To test the migration:

1. Run Flutter pub get to install dependencies:
   ```
   flutter pub get
   ```

2. Generate localization files:
   ```
   flutter gen-l10n
   ```

3. Run the app in different locales:
   ```
   flutter run --locale=en
   flutter run --locale=es
   ```

4. Verify that:
   - Both panels use Material 3 styling (cards with reduced elevation, updated typography)
   - All text is properly localized when switching languages
   - The UI respects the Material 3 color scheme
   - The panels function correctly for adding/removing items and images
"""

print(test_instructions)

## Summary
We've successfully migrated the CollectionPropertyPanel and ImagePropertyPanel to Material 3 design and integrated localization support:

1. Created new files with the 'm3_' prefix
2. Updated UI elements to follow Material 3 design guidelines
3. Extracted hardcoded strings into localization resources
4. Added support for multiple languages (English and Spanish as examples)
5. Provided integration and testing guidance

This migration enhances both the visual appearance of the application through Material 3 and improves accessibility and user experience through localization.